## Data Wrangling: Clean, Transform, Merge, Reshape

In [1]:
import pandas as pd

## Combining and merging data sets

### Database-style DataFrame merges

In [2]:
df1 = pd.DataFrame({'data1': range(7), 'key' : list('bbacaab')})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [3]:
df2 = pd.DataFrame({'data2': range(20,23), 'key': list('abd')})
df2

,data2,key
0,20,a
1,21,b
2,22,d


By default, .merge() performs an [inner join](https://www.w3schools.com/sql/sql_join.asp) between the DataFrames, using the common columns as keys.

In [4]:
df1.merge(df2)

,data1,key,data2
0,0,b,21
1,1,b,21
2,6,b,21
3,2,a,20
4,4,a,20
5,5,a,20


That means that it returns the cartesian product of the elements with common keys: if there are duplicates, it will return all the possible combinations:

In [6]:
df2_wdups = pd.DataFrame({'data2': range(20,24), 'key': list('abda')})
df1.merge(df2_wdups)

,data1,key,data2
0,0,b,21
1,1,b,21
2,6,b,21
3,2,a,20
4,2,a,23
5,4,a,20
6,4,a,23
7,5,a,20
8,5,a,23


If the columns to join on don't have the same name, or we want to join on the index of the DataFrames, we'll need to specify that.

In [8]:
df3 = pd.DataFrame({'data3': range(7), 'key3' : list('bbacaab')})
df4 = pd.DataFrame({'data4': range(20,23), 'key4' : list('abd')})

df3.merge(df4)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [9]:
df3.merge(df4, left_on='key3', right_on='key4')

,data3,key3,data4,key4
0,0,b,21,b
1,1,b,21,b
2,6,b,21,b
3,2,a,20,a
4,4,a,20,a
5,5,a,20,a


In [11]:
df1.merge(df2, how='outer')

,data1,key,data2
0,0.0,b,21.0
1,1.0,b,21.0
2,6.0,b,21.0
3,2.0,a,20.0
4,4.0,a,20.0
5,5.0,a,20.0
6,3.0,c,NaN
7,NaN,d,22.0


In [12]:
df1.merge(df2, how='left')

,data1,key,data2
0,0,b,21.0
1,1,b,21.0
2,2,a,20.0
3,3,c,NaN
4,4,a,20.0
5,5,a,20.0
6,6,b,21.0


If there are two columns with the same name that we do not join on, both will get transferred to the resulting DataFrame with a suffix. We can customize these suffixes.

In [17]:
df3['b'] = 'holi'
df4['new_col'] = 'adiosi'

df3

,data3,key3,new_col
0,0,b,holi
1,1,b,holi
2,2,a,holi
3,3,c,holi
4,4,a,holi
5,5,a,holi
6,6,b,holi


In [18]:
df3.merge(df4)

,data3,key3,new_col,data4,key4


In [19]:
df3.merge(df4, left_on='key3', right_on='key4')

,data3,key3,new_col_x,data4,key4,new_col_y
0,0,b,holi,21,b,adiosi
1,1,b,holi,21,b,adiosi
2,6,b,holi,21,b,adiosi
3,2,a,holi,20,a,adiosi
4,4,a,holi,20,a,adiosi
5,5,a,holi,20,a,adiosi


In [20]:
df3.merge(df4, left_on='key3', right_on='key4', suffixes=['_left', '_right'])

,data3,key3,new_col_left,data4,key4,new_col_right
0,0,b,holi,21,b,adiosi
1,1,b,holi,21,b,adiosi
2,6,b,holi,21,b,adiosi
3,2,a,holi,20,a,adiosi
4,4,a,holi,20,a,adiosi
5,5,a,holi,20,a,adiosi


In [22]:
[col + '_df3' for col in df3.columns]

['data3_df3', 'key3_df3', 'new_col_df3']

### Merging on index

In [25]:
df5 = pd.DataFrame({'holi' : range(4), 'children' : range(17,21)}, index = list('abcd'))
df5

,holi,children
a,0,17
b,1,18
c,2,19
d,3,20


In [27]:
df1.merge(df5, left_on='key', right_index=True)

,data1,key,holi,children
0,0,b,1,18
1,1,b,1,18
6,6,b,1,18
2,2,a,0,17
4,4,a,0,17
5,5,a,0,17
3,3,c,2,19


### Concatenating along an axis

In [31]:
pd.concat([df1, df5]) 

/home/dani/anaconda3/envs/master2019/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,children,data1,holi,key
0,NaN,0.0,NaN,b
1,NaN,1.0,NaN,b
2,NaN,2.0,NaN,a
3,NaN,3.0,NaN,c
4,NaN,4.0,NaN,a
5,NaN,5.0,NaN,a
6,NaN,6.0,NaN,b
a,17.0,NaN,0.0,NaN
b,18.0,NaN,1.0,NaN
c,19.0,NaN,2.0,NaN


In [36]:
s1 = pd.Series(range(7, 19, 2), index=list('abcdef'))
s2 = pd.Series(range(2, 18, 2), index=list('xyzwprom'))
s3 = pd.Series(range(100, 150, 10), index=list('acxy1'))

s1, s2, s3

(a     7
 b     9
 c    11
 d    13
 e    15
 f    17
 dtype: int64, x     2
 y     4
 z     6
 w     8
 p    10
 r    12
 o    14
 m    16
 dtype: int64, a    100
 c    110
 x    120
 y    130
 1    140
 dtype: int64)

In [38]:
pd.concat([s1, s2, s3], axis=1) 

/home/dani/anaconda3/envs/master2019/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1,2
1,NaN,NaN,140.0
a,7.0,NaN,100.0
b,9.0,NaN,NaN
c,11.0,NaN,110.0
d,13.0,NaN,NaN
e,15.0,NaN,NaN
f,17.0,NaN,NaN
m,NaN,16.0,NaN
o,NaN,14.0,NaN
p,NaN,10.0,NaN


#### Digression

Attention! Be careful not to reassign to reserved words or functions- you will overwrite the variable.

In [40]:
pd.DataFrame = df1
pd.DataFrame

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [41]:
df2 = pd.DataFrame({'data2': range(20,23), 'key': list('abd')})

TypeError: 'DataFrame' object is not callable

You can delete the overwritten variable, but you won't get back the original value. If it is an object or function from a module, you'll need to reload() the module, since Python doesn't load again an already imported module if you try to import it. reload() is useful also when you are actively developing your own module and want to load the latest definition of a function into memory.

In [43]:
del(pd.DataFrame)
pd.DataFrame

AttributeError: DataFrame

In [45]:
import pandas as pd
pd.DataFrame

AttributeError: module 'pandas' has no attribute 'DataFrame'

In [48]:
from importlib import reload

reload(pd)

<module 'pandas' from '/home/dani/anaconda3/envs/master2019/lib/python3.6/site-packages/pandas/__init__.py'>

In [49]:
pd.DataFrame

pandas.core.frame.DataFrame

## Data transformation

### Removing duplicates

In [63]:
dupped = pd.concat([df2,df2])
dupped

,data2,key
0,20,a
1,21,b
2,22,d
0,20,a
1,21,b
2,22,d


In [64]:
dupped.duplicated()

0    False
1    False
2    False
0     True
1     True
2     True
dtype: bool

In [65]:
dupped.duplicated(keep='last')

0     True
1     True
2     True
0    False
1    False
2    False
dtype: bool

In [70]:
merged = df1.merge(df3, left_on='key', right_on='key3')
merged.duplicated(subset=['data1', 'key'])

0     False
1      True
2      True
3     False
4      True
5      True
6     False
7      True
8      True
9     False
10     True
11     True
12    False
13     True
14     True
15    False
16     True
17     True
18    False
dtype: bool

In [72]:
merged.drop_duplicates(subset=['data1', 'key'], keep='last')

,data1,key,data3,key3,new_col
2,0,b,6,b,holi
5,1,b,6,b,holi
8,6,b,6,b,holi
11,2,a,5,a,holi
14,4,a,5,a,holi
17,5,a,5,a,holi
18,3,c,3,c,holi


In [73]:
merged.drop_duplicates(subset=['data1', 'key'], keep='first')

,data1,key,data3,key3,new_col
0,0,b,0,b,holi
3,1,b,0,b,holi
6,6,b,0,b,holi
9,2,a,2,a,holi
12,4,a,2,a,holi
15,5,a,2,a,holi
18,3,c,3,c,holi


### Renaming axis indexes

In [76]:
df2.index = list('xyz')
df2

,data2,key
x,20,a
y,21,b
z,22,d


### Discretization and binning

In [86]:
import random

ages = random.choices(range(1, 100), k=300)
cuts = pd.cut(ages, [0, 3, 12, 19, 35, 65, 100])

In [87]:
ages[:10]

[43, 7, 80, 6, 38, 23, 62, 67, 51, 9]

In [92]:
cuts.dtype

CategoricalDtype(categories=[(0, 3], (3, 12], (12, 19], (19, 35], (35, 65], (65, 100]]
              ordered=True)

In [89]:
cuts.value_counts()

(0, 3]         7
(3, 12]       36
(12, 19]      27
(19, 35]      43
(35, 65]      80
(65, 100]    107
dtype: int64

In [97]:
age_series = pd.Series(ages)
age_series.map(lambda age: 'child' if age < 18 else 'adult')[:10]

0    adult
1    child
2    adult
3    child
4    adult
5    adult
6    adult
7    adult
8    adult
9    child
dtype: object

## String manipulation

### String object methods

In [100]:
'holi'.upper()

'HOLI'

In [101]:
'holi'[:2]

'ho'

In [105]:
df3['new_col'] * 2

0    holiholi
1    holiholi
2    holiholi
3    holiholi
4    holiholi
5    holiholi
6    holiholi
Name: new_col, dtype: object

In [106]:
df3['new_col'].upper()

AttributeError: 'Series' object has no attribute 'upper'

### Vectorized string functions in pandas

[Vectorized string functions in pandas](https://pandas.pydata.org/pandas-docs/stable/text.html) are grouped within the .str attribute of Series and Indexes. They have the same names as the regular Python string functions, but work on Series of strings.

In [110]:
df3['data3'].str

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [107]:
df3['new_col'].str

In [124]:
df3['new_col'].str.capitalize()

0    Holi
1    Holi
2    Holi
3    Holi
4    Holi
5    Holi
6    Holi
Name: new_col, dtype: object

In [125]:
df3['new_col'].str.upper()

0    HOLI
1    HOLI
2    HOLI
3    HOLI
4    HOLI
5    HOLI
6    HOLI
Name: new_col, dtype: object

In [126]:
df3['new_col'].str.zfill(8)

0    0000holi
1    0000holi
2    0000holi
3    0000holi
4    0000holi
5    0000holi
6    0000holi
Name: new_col, dtype: object

In [127]:
df3['new_col'].str[:2]

0    ho
1    ho
2    ho
3    ho
4    ho
5    ho
6    ho
Name: new_col, dtype: object

In [118]:
import numpy as np

df3['semiempty'] = list('abcd') + [np.nan] * 3

In [122]:
df3['semiempty'].str